In [88]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

## Task1

빈 칸을 채워주세요!

단계별 output이 github 파일에는 남아있으니 그 output과 동일한 형태인지 확인하면서 진행해주시면 됩니다~

In [89]:
#1. 생성할 문장 데이터

sentence = ("Brick walls are there for a reason and you must not think "
            "that the brick walls aren't there to keep us out, but rather "
            "in this way that the brick walls are there to show us how badly we want things.")

In [90]:
#2. 문자 집합 만들기
world_set = list(set(sentence))

## 문제(1): 각 문자에 정수 인코딩 (공백도 하나의 원소로 포함)
vocab = {char: idx for idx, char in enumerate(world_set)}

In [91]:
print(vocab)

{'e': 0, '.': 1, 'u': 2, 'c': 3, 'g': 4, 'b': 5, 'i': 6, 'm': 7, 'd': 8, 'a': 9, 'B': 10, 'o': 11, 'n': 12, "'": 13, 'p': 14, 'r': 15, 's': 16, 'f': 17, ',': 18, 'w': 19, 'l': 20, 'y': 21, ' ': 22, 't': 23, 'k': 24, 'h': 25}


In [92]:
#3. 문자 집합 크기 확인

vocab_size = len(vocab)
print('문자 집합 크기 : {}'.format(vocab_size))

문자 집합 크기 : 26


In [93]:
#4. 하이퍼 파라미터 설정(자유롭게 수정해보세요!)

hidden_size = vocab_size # 같아야 하는 것 확인!
sequence_length = 10  # 너무 길거나 너무 짧게 잡으면 안됩니다!
learning_rate = 0.01

- hidden size와 vocab size가 같아야 하는 이유

추후에 원핫인코딩방식을 사용하기 때문

=> 표현하고 싶은 단어의 인덱스 위치에 1을 부여 / 다른 인덱스 위치에는 0을 부여 따라서 문자의 개수가 벡터의 크기가 되므로 둘을 일치시키는 것이 적절

[참고자료]

https://wikidocs.net/60853


In [94]:
#5. seqence 길이 단위 자르기

# 데이터 구성을 위한 리스트
x_data = []
y_data = []

## 문제(2): 반복문 내에서의 인덱싱을 사용하여 sequence_length 값 단위로 샘플을 잘라 데이터 만들기, y_str은 x_str은 한 칸씩 쉬프트된 sequnce

for i in range(0, len(sentence) - sequence_length):
  x_str = sentence[i:i+sequence_length]
  y_str = sentence[i+1 : i+sequence_length + 1]
  print(i, x_str, "->", y_str)

#x_str, y_str : 주어진 문장에서 잘라낸 일정 길이의 문자열
#현재 위치에서 sequence_length만큼의 문자열
#x_str, y_str을 순회하며 vocab 딕셔너리에 존재하는지 찾아서 리스트에 추가한것이 x_data, y_data(정수 인코딩된 값)

  # x_str과 y_str이 문자집합에 해당하는 인덱스를 각각 x_data, y_data에 append
  x_data.append([vocab[c] for c in x_str])
  y_data.append([vocab[d] for d in y_str])

0 Brick wall -> rick walls
1 rick walls -> ick walls 
2 ick walls  -> ck walls a
3 ck walls a -> k walls ar
4 k walls ar ->  walls are
5  walls are -> walls are 
6 walls are  -> alls are t
7 alls are t -> lls are th
8 lls are th -> ls are the
9 ls are the -> s are ther
10 s are ther ->  are there
11  are there -> are there 
12 are there  -> re there f
13 re there f -> e there fo
14 e there fo ->  there for
15  there for -> there for 
16 there for  -> here for a
17 here for a -> ere for a 
18 ere for a  -> re for a r
19 re for a r -> e for a re
20 e for a re ->  for a rea
21  for a rea -> for a reas
22 for a reas -> or a reaso
23 or a reaso -> r a reason
24 r a reason ->  a reason 
25  a reason  -> a reason a
26 a reason a ->  reason an
27  reason an -> reason and
28 reason and -> eason and 
29 eason and  -> ason and y
30 ason and y -> son and yo
31 son and yo -> on and you
32 on and you -> n and you 
33 n and you  ->  and you m
34  and you m -> and you mu
35 and you mu -> nd you mus
36

In [95]:
# 출력해서 한 칸씩 쉬프트된 것 확인하기!

print(x_data[0])
print(y_data[0])

[10, 15, 6, 3, 24, 22, 19, 9, 20, 20]
[15, 6, 3, 24, 22, 19, 9, 20, 20, 16]


In [96]:
##6. 입력 시퀀스에 대해 원핫인코딩 수행

## 문제(4) : x_data를 원핫인코딩 > numpy의 eye를 쓸 수 있지 않을까?
x_one_hot = [np.eye(vocab_size)[idx] for idx in x_data]

##7. 입력 데이터, 레이블데이터 텐서로 변환

## 문제(5) : x_one_hot과 y_data 텐서로 변환 : 둘 다 같은 형식의 텐서로 변환하면 될까?? (FloatTensor, LongTesor 중 맞는 것은?)
X = torch.FloatTensor(np.array(x_one_hot))
Y = torch.LongTensor(y_data)

numpy - eye

n*m 행렬을 만들어주며, 배열은 identity 행렬과 유사함.



```
In [2]:np.eye(N=3, M=5, # 총 행렬의 크기는 3x5 로 만들어 달라
               k=1, # identity matrix의 시작은 1열부터 시작하라
			   dtype=np.int8) # dtype은 int8로 설정
         
Out[2]:array([[0, 1, 0, 0, 0],
              [0, 0, 1, 0, 0],
              [0, 0, 0, 1, 0]],
              dtype=int8)
```



출처: https://coding-grandpa.tistory.com/entry/NumPy-identity-행렬-eye-diag-함수-이해하기-Creation-Function-III [개발자로 취직하기:티스토리]


보통 신경망 input은 float tensor, 손실함수와 비교되는 output은 long tensor로 만들어 줌(정수값)

In [97]:
##8. 크기 확인
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([188, 10, 26])
레이블의 크기 : torch.Size([188, 10])


In [98]:
##9.원핫인코딩 결과 샘플 확인하기
print(X[0])

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0.,

In [99]:
##10. 레이블 데이터 샘플 확인하기
print(Y[0])

tensor([15,  6,  3, 24, 22, 19,  9, 20, 20, 16])


In [100]:
len(x_one_hot[0][0]) #one hot encoding size

26

In [101]:
##11. RNN 모델 구현

##문제(6) : 기본 pytorch 인자 넣기 연습 + forward 채우기
### 조건 : rnn layer 2개 쌓기 + 마지막은 fc layer
### batch_fisrt 설정 필요할까? (유튜브 강의 참고)

class Net(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, layers):
        super(Net, self).__init__()
        self.rnn1 = torch.nn.RNN(input_size=input_dim, hidden_size=hidden_dim, num_layers=layers, batch_first=True)
        self.rnn2 = torch.nn.RNN(input_size=hidden_dim, hidden_size=hidden_dim, num_layers=layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        out, _ = self.rnn1(x)
        out, _ = self.rnn2(out)
        out = self.fc(out)
        return out

In [102]:
net = Net(vocab_size, hidden_size, 2)

In [103]:
##12. loss function
criterion = torch.nn.CrossEntropyLoss()

##13. optimizer
optimizer = optim.Adam(net.parameters(), learning_rate)

In [104]:
##14. 출력 크기 점검
outputs = net(X)
print(outputs.shape)

torch.Size([188, 10, 26])


In [105]:
##15. Training 시작

for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    ##문제(7) : outputs, Y 형태 그대로 넣으면 안되죠. view 함수를 이용해 loss값을 계산해봅시다.
    loss = criterion(outputs.view(-1, vocab_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([world_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += world_set[result[-1]]

    print(predict_str)

ppffffffffffffffffffffffffffffffffffffffffffffnffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff
n                                                                                                                                                                                                    
                                                                                                                                                                                                     
                                                                                                                                                                                                     
                                                                                                                                                                                                     
          

In [106]:
predict_str

' ere talls are there tot a te s t ari tht lats uht ther  that the brere talls are at there thes e  us  ut  but aather tr thes tat that the brere talls are there theso t us  ot brils t  tol  thes ta'

결과가 어떤가요?? 마지막 에폭의 문장이 그럴싸한가요?

## Task2

위 sentence는 제가 임의로 생성한 문장들입니다.

마음에 드시는 문구 가져오셔서 문장이 어떻게 생성되는지 확인해보세요!

영어가 아닌 한국어로 시도해보는 것도 좋겠죠?

수정이 많이 필요(토큰화 등) 할 수 있으나 한번 시도해보시는 것 권장드립니다 :)

위 베이스라인은 어디든 수정하셔도 좋고 조금 더 자연스러운 문장이 나올 수 있게 다양한 시도를 해보세요!

조건 : 문장 3개 이상, 연결성이 있는 문장을 " " 으로 구분하여 ( )에 넣기

과제 전체 참고 링크

https://wikidocs.net/64517

https://mr-doosun.tistory.com/25

##1. 불용어처리 + baseline

In [107]:
sentence = ("배가 고프다"
            "그래서 김치찌개를 먹을 것이다"
            "김치찌개랑 밥이랑 계란을 같이 먹어야지")

In [108]:
world_set = list(set(sentence))
vocab = {char: idx for idx, char in enumerate(world_set)}

In [109]:
print(vocab)

{'먹': 0, '고': 1, '치': 2, '을': 3, '계': 4, '프': 5, '개': 6, '란': 7, '야': 8, '를': 9, '같': 10, '이': 11, '어': 12, '찌': 13, '서': 14, '가': 15, '배': 16, '다': 17, '것': 18, '그': 19, '김': 20, '밥': 21, ' ': 22, '래': 23, '랑': 24, '지': 25}


적절하지 않음 -> 형태소 토큰화

In [117]:
pip install nltk

In [110]:
!pip install konlpy
!pip install mecab-python
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

mecab-ko is already installed
mecab-ko-dic is already installed
mecab-python is already installed
Done.


In [125]:
from konlpy.tag import Mecab
tokenizer = Mecab()
print(tokenizer.morphs(sentence))

['배', '가', '고프', '다', '그래서', '김치찌개', '를', '먹', '을', '것', '이', '다', '김치찌개', '랑', '밥', '이랑', '계란', '을', '같이', '먹', '어야지']


In [130]:
tokenized = tokenizer.morphs(sentence)

In [131]:
# 불용어 정의
stopwords=['가','다','를','을','이','다','랑','이랑','어야지','']

In [132]:
tokenized = [word for word in tokenized if word not in stopwords] #불용어 제거

In [133]:
print(tokenized)

['배', '고프', '그래서', '김치찌개', '먹', '것', '김치찌개', '밥', '계란', '같이', '먹']


In [137]:
#단어 집합 생성
word_to_idx = {word: idx for idx, word in enumerate(set(tokenized))} #tokenized에 포함된 단어들을중복 제거 -> 단어를 key, 인덱스를 값으로 사용하여 매핑
idx_to_word = {idx: word for word, idx in word_to_idx.items()} #word to idx 키와 값 순서 바꿔서 변환 매핑

In [140]:
#데이터 생성
input= []
target = []

sequence_length = 3 #문장이 짧아서 짧게 설정
learning_rate = 0.01
vocab_size = len(word_to_idx)
hidden_size = vocab_size

for i in range(len(tokenized) - sequence_length):
  input_seq = tokenized[i:i+sequence_length]
  target_seq = tokenized[i+1:i+sequence_length+1]

  input.append([word_to_idx[word] for word in input_seq])
  target.append([word_to_idx[word] for word in target_seq])

In [142]:
input = np.array(input)
target = np.array(target)

#one-hot encoding
def one_hot_encoding(data,num_classes):
  num_data = data.shape[0]
  num_seq = data.shape[1]
  one_hot = np.zeros((num_data,num_seq,num_classes), dtype=np.float32)
  for i in range(num_data):
    for j in range(num_seq):
      one_hot[i,j,data[i,j]] = 1
  return one_hot

In [143]:
x_one_hot = one_hot_encoding(input, vocab_size)

X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(target)

In [145]:
x_one_hot[0]

array([[0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0.]], dtype=float32)

In [146]:
#RNN (맨 위 베이스라인 참고)

class Net(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, layers):
        super(Net, self).__init__()
        self.rnn1 = torch.nn.RNN(input_size=input_dim, hidden_size=hidden_dim, num_layers=layers, batch_first=True)
        self.rnn2 = torch.nn.RNN(input_size=hidden_dim, hidden_size=hidden_dim, num_layers=layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        out, _ = self.rnn1(x)
        out, _ = self.rnn2(out)
        out = self.fc(out)
        return out

In [150]:
net = Net(vocab_size, hidden_size, 2) #layer=2개

In [151]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

In [152]:
outputs = net(X)
print(outputs.shape)

torch.Size([8, 3, 9])


In [153]:
#학습
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    loss = criterion(outputs.view(-1, vocab_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([world_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += world_set[result[-1]]

    print(predict_str)

야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야개야야
야개야야야야먹개야먹
야개야먹야야먹개야먹
프개야먹야야먹개야먹
프개야먹먹야먹개야먹
프란야먹먹야먹개야먹
프란야먹먹야프개야먹
프란야먹먹야프개야먹
프란야먹먹야프개야먹
프란야먹프야프개야먹
프란야먹프야프개야먹
프란야먹개야프개야먹
프란야먹개야프개야먹
프란야먹개야프개야먹
프란야먹개야프개을먹
계란야먹개야프개을먹
계란야먹개야프개을먹
계란야먹개야프개을먹
계란야먹개야프개을먹
계란야먹개야프개을먹
계란야먹개야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹

In [154]:
predict_str

'계란야먹치야프개을먹'

불용어 처리를 안하고 해보자.

##2. 불용어 X + baseline

In [155]:
sentence = ("배가 고프다"
            "그래서 김치찌개를 먹을 것이다"
            "김치찌개랑 밥이랑 계란을 같이 먹어야지")

In [156]:
world_set = list(set(sentence))
vocab = {char: idx for idx, char in enumerate(world_set)}

In [157]:
print(vocab)

{'먹': 0, '고': 1, '치': 2, '을': 3, '계': 4, '프': 5, '개': 6, '란': 7, '야': 8, '를': 9, '같': 10, '이': 11, '어': 12, '찌': 13, '서': 14, '가': 15, '배': 16, '다': 17, '것': 18, '그': 19, '김': 20, '밥': 21, ' ': 22, '래': 23, '랑': 24, '지': 25}


적절하지 않음 -> 형태소 토큰화

In [158]:
from konlpy.tag import Mecab
tokenizer = Mecab()
print(tokenizer.morphs(sentence))

['배', '가', '고프', '다', '그래서', '김치찌개', '를', '먹', '을', '것', '이', '다', '김치찌개', '랑', '밥', '이랑', '계란', '을', '같이', '먹', '어야지']


In [159]:
tokenized = tokenizer.morphs(sentence)

In [160]:
print(tokenized)

['배', '가', '고프', '다', '그래서', '김치찌개', '를', '먹', '을', '것', '이', '다', '김치찌개', '랑', '밥', '이랑', '계란', '을', '같이', '먹', '어야지']


In [161]:
#단어 집합 생성
word_to_idx = {word: idx for idx, word in enumerate(set(tokenized))} #tokenized에 포함된 단어들을중복 제거 -> 단어를 key, 인덱스를 값으로 사용하여 매핑
idx_to_word = {idx: word for word, idx in word_to_idx.items()} #word to idx 키와 값 순서 바꿔서 변환 매핑

In [162]:
#데이터 생성
input= []
target = []

sequence_length = 3 #문장이 짧아서 짧게 설정
learning_rate = 0.01
vocab_size = len(word_to_idx)
hidden_size = vocab_size

for i in range(len(tokenized) - sequence_length):
  input_seq = tokenized[i:i+sequence_length]
  target_seq = tokenized[i+1:i+sequence_length+1]

  input.append([word_to_idx[word] for word in input_seq])
  target.append([word_to_idx[word] for word in target_seq])

In [163]:
input = np.array(input)
target = np.array(target)

#one-hot encoding
def one_hot_encoding(data,num_classes):
  num_data = data.shape[0]
  num_seq = data.shape[1]
  one_hot = np.zeros((num_data,num_seq,num_classes), dtype=np.float32)
  for i in range(num_data):
    for j in range(num_seq):
      one_hot[i,j,data[i,j]] = 1
  return one_hot

In [164]:
x_one_hot = one_hot_encoding(input, vocab_size)

X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(target)

In [165]:
x_one_hot[0]

array([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.]], dtype=float32)

In [166]:
#RNN (맨 위 베이스라인 참고)

class Net(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, layers):
        super(Net, self).__init__()
        self.rnn1 = torch.nn.RNN(input_size=input_dim, hidden_size=hidden_dim, num_layers=layers, batch_first=True)
        self.rnn2 = torch.nn.RNN(input_size=hidden_dim, hidden_size=hidden_dim, num_layers=layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        out, _ = self.rnn1(x)
        out, _ = self.rnn2(out)
        out = self.fc(out)
        return out

In [167]:
net = Net(vocab_size, hidden_size, 2) #layer=2개

In [168]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

In [169]:
outputs = net(X)
print(outputs.shape)

torch.Size([18, 3, 17])


In [170]:
#학습
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    loss = criterion(outputs.view(-1, vocab_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([world_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += world_set[result[-1]]

    print(predict_str)

를를를를를를를를를를를를를를를를를를를를
를를를를를를를를를를를를를를를를를를를를
를배배배배배배배배배배배배배배배배배배배
배배배배배배배배배배배배배배배배배배배배
배배배배배배배배배배배배배배배배배배배배
배배배배배배배배배배배배배배배배배배배배
배배배배배배배배배배을배배배배배배배고을
배고을배배배을배배고을배배을배을배배고을
배고을배을배을배배고을배고을배을배고고을
배고을배을고을배배고을배이을배을배고고을
을고을배을배을배배고을배이을배을배고고을
을고을배이배을배배프을배이을배을배고고을
을고를배이배을배계프을배프을배를배고고을
을란를같배배고배계프을배프을배를배고고을
을란을같배어고배계프을배프을배를배개고을
을란을같배어고배계프을배프을배를배개고을
을란을같이어고배계프을배프을찌를배개고을
을란을같이어고배계프을배프을찌를배개고을
을란을같이어고배계프을배프을찌를배개고을
을란을같이어고배계프을배프을찌를배개고을
을란을같이어고배계프을배프을찌를배개고을
을란을같이어고배계프을배프을찌를배개고을
을란을같이어고배계프을배프을찌를배개고을
을란을같이어고배계프을이프을찌를배개고을
을란을같이어고배계프을이프야찌를배개고을
을란을같이어고배계프을이가야찌를배개고을
을란을같이어고배계프을이가야찌를배개고을
을란을같이어고배계프을이가야찌를배개고을
을란을같이어고배계프을이가야찌를배개고을
을란을같이어고배계프을이가야찌를배개고을
을란을같이어고배계프을이가야찌를배개고을
을란을같이어고배계프을이가야찌를배개고을
을란을같이어고배계프을이가야찌를배개고을
을란을같이어고배계프을이가야찌를배개고을
먹란을같이어고배계프을이가야찌를배개고을
먹란을같이어고배계프을이가야찌를배개고을
먹란을같이어고배계프을이가야찌를배개고을
먹란을같이어고배계프을이가야찌를배개고서
먹란을같이어고배계프을이가야찌를배개고서
먹란을같이어고배계프을이가야찌를배개고서
먹란을같이어고배계프을이가야찌를배개고서
먹란을같이어고배계프을이가야찌를배개고서
먹란을같이어고배계프을이가야찌를배개고서
먹란을같이어고배계프을이가야찌를배개고서
먹란을같이어고배계프을이가야찌를배개고서
먹란을같이어고배계프을이가야찌를배개고서
먹란을같이어고배계프을이가야찌를배개고서
먹란을같이어고배계프을이가

In [171]:
predict_str

'먹란을같이어고배계프을이가야찌를배개고서'

##3. 불용어처리 + baseline 수정 + sequence length 늘림

In [188]:
sentence = ("배가 고프다"
            "그래서 김치찌개를 먹을 것이다"
            "김치찌개랑 밥이랑 계란을 같이 먹어야지")

In [189]:
world_set = list(set(sentence))
vocab = {char: idx for idx, char in enumerate(world_set)}

In [190]:
print(vocab)

{'먹': 0, '고': 1, '치': 2, '을': 3, '계': 4, '프': 5, '개': 6, '란': 7, '야': 8, '를': 9, '같': 10, '이': 11, '어': 12, '찌': 13, '서': 14, '가': 15, '배': 16, '다': 17, '것': 18, '그': 19, '김': 20, '밥': 21, ' ': 22, '래': 23, '랑': 24, '지': 25}


적절하지 않음 -> 형태소 토큰화

In [191]:
from konlpy.tag import Mecab
tokenizer = Mecab()
print(tokenizer.morphs(sentence))

['배', '가', '고프', '다', '그래서', '김치찌개', '를', '먹', '을', '것', '이', '다', '김치찌개', '랑', '밥', '이랑', '계란', '을', '같이', '먹', '어야지']


In [192]:
tokenized = tokenizer.morphs(sentence)

In [193]:
# 불용어 정의
stopwords=['가','다','를','을','이','다','랑','이랑','어야지','']

In [194]:
tokenized = [word for word in tokenized if word not in stopwords] #불용어 제거

In [195]:
print(tokenized)

['배', '고프', '그래서', '김치찌개', '먹', '것', '김치찌개', '밥', '계란', '같이', '먹']


In [196]:
#단어 집합 생성
word_to_idx = {word: idx for idx, word in enumerate(set(tokenized))} #tokenized에 포함된 단어들을중복 제거 -> 단어를 key, 인덱스를 값으로 사용하여 매핑
idx_to_word = {idx: word for word, idx in word_to_idx.items()} #word to idx 키와 값 순서 바꿔서 변환 매핑

In [197]:
#데이터 생성
input= []
target = []

sequence_length = 5 #문장이 짧아서 짧게 설정 -> 늘려줌
learning_rate = 0.01
vocab_size = len(word_to_idx)
hidden_size = vocab_size

for i in range(len(tokenized) - sequence_length):
  input_seq = tokenized[i:i+sequence_length]
  target_seq = tokenized[i+1:i+sequence_length+1]

  input.append([word_to_idx[word] for word in input_seq])
  target.append([word_to_idx[word] for word in target_seq])

In [198]:
input = np.array(input)
target = np.array(target)

#one-hot encoding
def one_hot_encoding(data,num_classes):
  num_data = data.shape[0]
  num_seq = data.shape[1]
  one_hot = np.zeros((num_data,num_seq,num_classes), dtype=np.float32)
  for i in range(num_data):
    for j in range(num_seq):
      one_hot[i,j,data[i,j]] = 1
  return one_hot

In [199]:
x_one_hot = one_hot_encoding(input, vocab_size)

X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(target)

In [200]:
x_one_hot[0]

array([[0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [201]:
#RNN (맨 위 베이스라인 참고)

class Net(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, layers):
        super(Net, self).__init__()
        self.rnn1 = torch.nn.RNN(input_size=input_dim, hidden_size=hidden_dim, num_layers=layers, batch_first=True)
        self.rnn2 = torch.nn.RNN(input_size=hidden_dim, hidden_size=hidden_dim, num_layers=layers, batch_first=True)
        self.rnn3 = torch.nn.RNN(input_size=hidden_dim, hidden_size=hidden_dim, num_layers=layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        out, _ = self.rnn1(x)
        out, _ = self.rnn2(out)
        out, _ = self.rnn3(out)
        out = self.fc(out)
        return out

In [203]:
net = Net(vocab_size, hidden_size, 3) #layer=3개

In [204]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

In [205]:
outputs = net(X)
print(outputs.shape)

torch.Size([6, 5, 9])


In [206]:
#학습
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    loss = criterion(outputs.view(-1, vocab_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([world_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += world_set[result[-1]]

    print(predict_str)

프고고고고고고고고고
프야야야고고고고고고
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야먹야야먹야
야야치먹치야야야먹야
야야치먹치야야치야야
치야치먹치야야치야먹
치야치먹치야야치야먹
치야치먹치야야치야먹
치먹치먹치야먹개야먹
치먹치먹치야야개야먹
치먹치먹치야야개야먹
치먹치먹치야먹개야먹
치먹치먹치야먹개야먹
치먹치먹치야프개야먹
치먹치먹치야프개야먹
야먹치먹치야프개개먹
야먹치먹치야프개야먹
야먹치먹치야프개개먹
야먹치먹치야프개개먹
야먹치먹치야프개을먹
야먹치먹치야프개을먹
야먹치먹치야프개을먹
야먹치먹치야프개을먹
야먹치먹치야프개을먹
야먹치먹치야프개을먹
야먹치먹치야프개을먹
야먹치먹치야프개을먹
야먹치먹치야프개을먹
야먹치먹치야프개을먹
야먹치먹치야프개을먹
야먹치먹치야프개을먹
야먹치먹치야프개을먹
야프치먹치야프개을먹
야먹치먹치야프개을먹
야먹치먹치야프개을먹
야먹치먹치야프개을먹
야먹치먹치야프개을먹
야먹치먹치야프개을먹
야먹치먹치야프개을먹
야먹치먹치야프개을먹
야먹치먹치야프개을먹
야먹치먹치야프개을먹
야먹치먹치야프개을먹
야먹치먹치야프개을먹
야먹치먹치야프개을먹
야먹치먹치야프개을먹
야먹치먹치야프개을먹
야먹치먹치야프개을먹
야먹치먹치야프개을먹
야먹치먹치야프개을먹
야먹치먹치야프개을먹
야먹치먹치야프개을먹
야먹치먹치야프개을먹
야먹치먹치야프개을먹
야프치먹치야프개을먹
야프치먹치야프개을먹
야프치먹치야프개을먹
야먹치먹치야프개을먹
야란치먹치야프개을먹
야란치먹치야프개을먹
야란치먹치야프개을먹
야프치먹치야프개을먹
야프치먹치야프개을먹
야프치먹치야프개을먹
야란치먹치야프개을먹
야란치먹치야프개을먹
야란치먹치야프개을먹
야란치먹치야프개을먹
야프치먹치야프개을먹
야프치먹치야프개을먹
야란치먹치야프개을먹

In [207]:
predict_str

'야프치먹치야프개을먹'

##4. 불용어X + baseline 수정 + sequence length 늘림

In [208]:
sentence = ("배가 고프다"
            "그래서 김치찌개를 먹을 것이다"
            "김치찌개랑 밥이랑 계란을 같이 먹어야지")

In [209]:
world_set = list(set(sentence))
vocab = {char: idx for idx, char in enumerate(world_set)}

In [210]:
print(vocab)

{'먹': 0, '고': 1, '치': 2, '을': 3, '계': 4, '프': 5, '개': 6, '란': 7, '야': 8, '를': 9, '같': 10, '이': 11, '어': 12, '찌': 13, '서': 14, '가': 15, '배': 16, '다': 17, '것': 18, '그': 19, '김': 20, '밥': 21, ' ': 22, '래': 23, '랑': 24, '지': 25}


적절하지 않음 -> 형태소 토큰화

In [211]:
from konlpy.tag import Mecab
tokenizer = Mecab()
print(tokenizer.morphs(sentence))

['배', '가', '고프', '다', '그래서', '김치찌개', '를', '먹', '을', '것', '이', '다', '김치찌개', '랑', '밥', '이랑', '계란', '을', '같이', '먹', '어야지']


In [212]:
tokenized = tokenizer.morphs(sentence)

In [213]:
print(tokenized)

['배', '가', '고프', '다', '그래서', '김치찌개', '를', '먹', '을', '것', '이', '다', '김치찌개', '랑', '밥', '이랑', '계란', '을', '같이', '먹', '어야지']


In [214]:
#단어 집합 생성
word_to_idx = {word: idx for idx, word in enumerate(set(tokenized))} #tokenized에 포함된 단어들을중복 제거 -> 단어를 key, 인덱스를 값으로 사용하여 매핑
idx_to_word = {idx: word for word, idx in word_to_idx.items()} #word to idx 키와 값 순서 바꿔서 변환 매핑

In [215]:
#데이터 생성
input= []
target = []

sequence_length = 5 #문장이 짧아서 짧게 설정 -> 늘려줌
learning_rate = 0.01
vocab_size = len(word_to_idx)
hidden_size = vocab_size

for i in range(len(tokenized) - sequence_length):
  input_seq = tokenized[i:i+sequence_length]
  target_seq = tokenized[i+1:i+sequence_length+1]

  input.append([word_to_idx[word] for word in input_seq])
  target.append([word_to_idx[word] for word in target_seq])

In [216]:
input = np.array(input)
target = np.array(target)

#one-hot encoding
def one_hot_encoding(data,num_classes):
  num_data = data.shape[0]
  num_seq = data.shape[1]
  one_hot = np.zeros((num_data,num_seq,num_classes), dtype=np.float32)
  for i in range(num_data):
    for j in range(num_seq):
      one_hot[i,j,data[i,j]] = 1
  return one_hot

In [217]:
x_one_hot = one_hot_encoding(input, vocab_size)

X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(target)

In [218]:
x_one_hot[0]

array([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0.]], dtype=float32)

In [219]:
#RNN (맨 위 베이스라인 참고)

class Net(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, layers):
        super(Net, self).__init__()
        self.rnn1 = torch.nn.RNN(input_size=input_dim, hidden_size=hidden_dim, num_layers=layers, batch_first=True)
        self.rnn2 = torch.nn.RNN(input_size=hidden_dim, hidden_size=hidden_dim, num_layers=layers, batch_first=True)
        self.rnn3 = torch.nn.RNN(input_size=hidden_dim, hidden_size=hidden_dim, num_layers=layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        out, _ = self.rnn1(x)
        out, _ = self.rnn2(out)
        out, _ = self.rnn3(out)
        out = self.fc(out)
        return out

In [220]:
net = Net(vocab_size, hidden_size, 3) #layer=3개

In [221]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

In [223]:
outputs = net(X)
print(outputs.shape)

torch.Size([16, 5, 17])


In [224]:
#학습
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    loss = criterion(outputs.view(-1, vocab_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([world_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += world_set[result[-1]]

    print(predict_str)

란가을을을을을을을을을을을을을을을을을을
을을을을을을을을을을을을을을을을을을을을
을을이이이이이이이이이이이이이이이이이이
을을이이이이이이이이이이이이이이이이이이
을이이이이이이이이이이이이이이이이이이이
을이이이이이이이이이이이이이이이이이이이
이이이이이이이이이이이이이이이이이이이이
이이이배배배이배배배이배배이배배배배배배
이이이배배배고배배배이배배고배배배고이배
이이이이이배고배배고이이배고배배배고고이
을이이이이배고배배고이이배고배배배고고이
을을을이이배고배배고을이배고배배배고고을
을을을이이배고배고고을이배야배배배고고을
을을을이이배야배야을을이배야배를배고고을
을을을이이배야배야을을이배야배를배개개을
을을을이이배야배야을을이배야배를배개개을
을을을이이배야배야을을이배야배를배개개을
을을을이이배야배야을을이배야배를배개개을
을을을이이배야배야을을이배야배를배개고을
을을을이이어야배를을을이어야배를배개고을
을을을이이배야배를을을이어야배를배개고을
을을을이이어야배를을을이어야배를배개고을
을을을이이어야배를프을이어야배를배개고을
을을을이이어야배계프을이어야배를배개고을
을을을이이어야배계프을이어야배를배개고야
을을을이이어야배계프을이어야배를배개고을
을을을이이어야배계프을이어야배를배개고을
을을을이이어고배계프을이가야배를배개고고
을을을이이가고배계프을이가야배를배개고을
을을을이이가고배계프을이가야배를배개고을
을을을이이가고배계프을이가야배를배개고고
을을을이이가고배계프을이가야배를배개고고
을을을이이가고배계프을이가야배를배개고을
을을을이이가고배계프을이가야배를배개고을
을을을이이가고배계프을이가야배를배개고고
을을을이이가고배계프을이가야찌를배개고고
을을을이이어고배계프을이가야찌를배개고을
을을을이이어고배계프을이가야찌를배개고서
을을을이이어고배계프을이가야찌를배개고고
을을을이이어고배계프을이가야찌를배개고고
을을을이이어고배계프을이가야찌를배개고고
을을을이이어고배계프을이어야찌를배개고서
을을을같이어고배계프을이어야찌를배개고서
을을을같이어고배계프을이어야찌를배개고서
을을을이이어고배계프을이어야찌를배개고서
야을을같이어고배계프을이가야찌를배개고서
야을을같이어고배계프을이가야찌를배개고서
을을을같이어고배계프을이가

In [225]:
predict_str

'먹란을같이어고배계프을이가야찌를배개고서'

##5. 불용어X + baseline 수정 + sequence length 늘림 + lr 0.05

In [226]:
sentence = ("배가 고프다"
            "그래서 김치찌개를 먹을 것이다"
            "김치찌개랑 밥이랑 계란을 같이 먹어야지")

In [227]:
world_set = list(set(sentence))
vocab = {char: idx for idx, char in enumerate(world_set)}

In [228]:
print(vocab)

{'먹': 0, '고': 1, '치': 2, '을': 3, '계': 4, '프': 5, '개': 6, '란': 7, '야': 8, '를': 9, '같': 10, '이': 11, '어': 12, '찌': 13, '서': 14, '가': 15, '배': 16, '다': 17, '것': 18, '그': 19, '김': 20, '밥': 21, ' ': 22, '래': 23, '랑': 24, '지': 25}


적절하지 않음 -> 형태소 토큰화

In [229]:
from konlpy.tag import Mecab
tokenizer = Mecab()
print(tokenizer.morphs(sentence))

['배', '가', '고프', '다', '그래서', '김치찌개', '를', '먹', '을', '것', '이', '다', '김치찌개', '랑', '밥', '이랑', '계란', '을', '같이', '먹', '어야지']


In [230]:
tokenized = tokenizer.morphs(sentence)

In [231]:
print(tokenized)

['배', '가', '고프', '다', '그래서', '김치찌개', '를', '먹', '을', '것', '이', '다', '김치찌개', '랑', '밥', '이랑', '계란', '을', '같이', '먹', '어야지']


In [232]:
#단어 집합 생성
word_to_idx = {word: idx for idx, word in enumerate(set(tokenized))} #tokenized에 포함된 단어들을중복 제거 -> 단어를 key, 인덱스를 값으로 사용하여 매핑
idx_to_word = {idx: word for word, idx in word_to_idx.items()} #word to idx 키와 값 순서 바꿔서 변환 매핑

In [234]:
#데이터 생성
input= []
target = []

sequence_length = 5 #문장이 짧아서 짧게 설정 -> 늘려줌
learning_rate = 0.05
vocab_size = len(word_to_idx)
hidden_size = vocab_size

for i in range(len(tokenized) - sequence_length):
  input_seq = tokenized[i:i+sequence_length]
  target_seq = tokenized[i+1:i+sequence_length+1]

  input.append([word_to_idx[word] for word in input_seq])
  target.append([word_to_idx[word] for word in target_seq])

In [235]:
input = np.array(input)
target = np.array(target)

#one-hot encoding
def one_hot_encoding(data,num_classes):
  num_data = data.shape[0]
  num_seq = data.shape[1]
  one_hot = np.zeros((num_data,num_seq,num_classes), dtype=np.float32)
  for i in range(num_data):
    for j in range(num_seq):
      one_hot[i,j,data[i,j]] = 1
  return one_hot

In [236]:
x_one_hot = one_hot_encoding(input, vocab_size)

X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(target)

In [237]:
x_one_hot[0]

array([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0.]], dtype=float32)

In [238]:
#RNN (맨 위 베이스라인 참고)

class Net(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, layers):
        super(Net, self).__init__()
        self.rnn1 = torch.nn.RNN(input_size=input_dim, hidden_size=hidden_dim, num_layers=layers, batch_first=True)
        self.rnn2 = torch.nn.RNN(input_size=hidden_dim, hidden_size=hidden_dim, num_layers=layers, batch_first=True)
        self.rnn3 = torch.nn.RNN(input_size=hidden_dim, hidden_size=hidden_dim, num_layers=layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        out, _ = self.rnn1(x)
        out, _ = self.rnn2(out)
        out, _ = self.rnn3(out)
        out = self.fc(out)
        return out

In [239]:
net = Net(vocab_size, hidden_size, 3) #layer=3개

In [240]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

In [241]:
outputs = net(X)
print(outputs.shape)

torch.Size([16, 5, 17])


In [242]:
#학습
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    loss = criterion(outputs.view(-1, vocab_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([world_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += world_set[result[-1]]

    print(predict_str)

을을을을을을을을을을을을을을을을을을을을
을을이를를이를이이를이를이이를이이이이이
을을이이이이이이이이이이이이이이이이이이
배이고고고고고고고고고고고고고고고고고고
배배배배배배배배배배배배배배배배배배배배
배배배배배배배배배배배배배배배배배배배배
를배배이이이이이이이이이이이이이이이이이
을을이이이이이이이이이이이이이이이이이이
을이이이이고이이이이이이이이이이이이이이
을이을를이고고고을이고이고고을을이이고을
이이배를이고고고을이을이고고배을이고고을
을이을을이배고을을이을이고을배을이고고을
을이을을이배이을이배이이고이배이배고을을
을이찌을이배이배이찌을이가찌을을이가고찌
을이찌을이가고찌이찌을이가고찌을이이가찌
을이가찌을이고찌계찌을이이고찌찌을이가찌
을이프을이가고프계프을이이고프을이이가찌
을이프을이야고프을이을이가찌프을이가고찌
을이배를이가고프를배를이가고서를이이가서
을이배를이개고프를배를배이어서를배이어서
을이배를이어고배를배를이가고서를이가고서
을란찌을이어고배를배를이어고서를배개어서
을란찌를이어고찌계배을이가어찌를배개어찌
을이찌을이어고찌계배을이어고찌를배개어찌
을가찌을이어고배고배을이어고찌을배개어찌
을가배을이어고배계배을이어고배을배개어찌
을가배을이어고배계배을이어고배을배개어서
을이어을이어고배계배을이어고배를배개어서
을이어을이어고배계배을이가고배를배개가서
어계프을이어고배계배을이가고찌를배개가서
야야찌을이어계배계배을이가고찌를배개가찌
가야찌을이가고배계배을이가고찌를배개가찌
가야찌을이가고배계프을이가고찌를배개가찌
란고배같이어고배계프계이가고찌배배개가찌
이고찌같이어고배이배을이가고찌배배이가찌
이이배같이어고배이프을이가고배같배개가찌
이이배을이가고프고프을이가고배같배개가서
이배을배개가이프고프을이가야찌를이가가서
이고찌를이가계프고프을이가야서를배개가서
이어찌를이고야찌이프을배개고서를배이고서
을어서를을이야찌계이찌같배이야어을배이야
을어찌을이어야찌계프을같이야찌같배를어서
을어어찌계어야찌계프찌배이야찌배을계어찌
계프을배계어야배계프배배가야을배배계어찌
계프계배이어고배이어배배가계계프배이어찌
고배계배이고고배이고배배가고찌찌배이고찌
고찌계배이고고배이고배이가고찌찌배이고찌
고배계배이고고배이고배이가

In [243]:
predict_str

'을계을배개야고배개프서이가야찌를배개프서'

##6. 불용어X + baseline 수정 + sequence length 늘림 + lr 0.001

In [244]:
sentence = ("배가 고프다"
            "그래서 김치찌개를 먹을 것이다"
            "김치찌개랑 밥이랑 계란을 같이 먹어야지")

In [245]:
world_set = list(set(sentence))
vocab = {char: idx for idx, char in enumerate(world_set)}

In [246]:
print(vocab)

{'먹': 0, '고': 1, '치': 2, '을': 3, '계': 4, '프': 5, '개': 6, '란': 7, '야': 8, '를': 9, '같': 10, '이': 11, '어': 12, '찌': 13, '서': 14, '가': 15, '배': 16, '다': 17, '것': 18, '그': 19, '김': 20, '밥': 21, ' ': 22, '래': 23, '랑': 24, '지': 25}


적절하지 않음 -> 형태소 토큰화

In [247]:
from konlpy.tag import Mecab
tokenizer = Mecab()
print(tokenizer.morphs(sentence))

['배', '가', '고프', '다', '그래서', '김치찌개', '를', '먹', '을', '것', '이', '다', '김치찌개', '랑', '밥', '이랑', '계란', '을', '같이', '먹', '어야지']


In [248]:
tokenized = tokenizer.morphs(sentence)

In [249]:
print(tokenized)

['배', '가', '고프', '다', '그래서', '김치찌개', '를', '먹', '을', '것', '이', '다', '김치찌개', '랑', '밥', '이랑', '계란', '을', '같이', '먹', '어야지']


In [250]:
#단어 집합 생성
word_to_idx = {word: idx for idx, word in enumerate(set(tokenized))} #tokenized에 포함된 단어들을중복 제거 -> 단어를 key, 인덱스를 값으로 사용하여 매핑
idx_to_word = {idx: word for word, idx in word_to_idx.items()} #word to idx 키와 값 순서 바꿔서 변환 매핑

In [252]:
#데이터 생성
input= []
target = []

sequence_length = 5 #문장이 짧아서 짧게 설정 -> 늘려줌
learning_rate = 0.001
vocab_size = len(word_to_idx)
hidden_size = vocab_size

for i in range(len(tokenized) - sequence_length):
  input_seq = tokenized[i:i+sequence_length]
  target_seq = tokenized[i+1:i+sequence_length+1]

  input.append([word_to_idx[word] for word in input_seq])
  target.append([word_to_idx[word] for word in target_seq])

In [253]:
input = np.array(input)
target = np.array(target)

#one-hot encoding
def one_hot_encoding(data,num_classes):
  num_data = data.shape[0]
  num_seq = data.shape[1]
  one_hot = np.zeros((num_data,num_seq,num_classes), dtype=np.float32)
  for i in range(num_data):
    for j in range(num_seq):
      one_hot[i,j,data[i,j]] = 1
  return one_hot

In [254]:
x_one_hot = one_hot_encoding(input, vocab_size)

X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(target)

In [255]:
x_one_hot[0]

array([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0.]], dtype=float32)

In [256]:
#RNN (맨 위 베이스라인 참고)

class Net(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, layers):
        super(Net, self).__init__()
        self.rnn1 = torch.nn.RNN(input_size=input_dim, hidden_size=hidden_dim, num_layers=layers, batch_first=True)
        self.rnn2 = torch.nn.RNN(input_size=hidden_dim, hidden_size=hidden_dim, num_layers=layers, batch_first=True)
        self.rnn3 = torch.nn.RNN(input_size=hidden_dim, hidden_size=hidden_dim, num_layers=layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        out, _ = self.rnn1(x)
        out, _ = self.rnn2(out)
        out, _ = self.rnn3(out)
        out = self.fc(out)
        return out

In [257]:
net = Net(vocab_size, hidden_size, 3) #layer=3개

In [258]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

In [259]:
outputs = net(X)
print(outputs.shape)

torch.Size([16, 5, 17])


In [260]:
#학습
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    loss = criterion(outputs.view(-1, vocab_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([world_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += world_set[result[-1]]

    print(predict_str)

이이계이이이이이이이이이이이이이이이이이
이이계이이이이이이이이이이이이이이이이이
이이계계이이이이이이이이이이이이이이이이
이이계계이이이이이이이이이이이이이이이이
이이계계이이이이계이이이이이이이이이계이
이이계계계계계계계계계계계계계계계계계계
이이계계계계계계계계계계계계계계계계계계
이이계계계계계계계계계계계계계계계계계계
이이계계계계계계계계계계계계계계계계계계
이이계계계계계계계계계계계계계계계계계계
이이계계계계계계계계계계계계계계계계계계
이이계계계계계계계계계계계계계계계계계계
이이계계계계계계계계계계계계계계계계계계
이이계계계계계계계계계계계계계계계계계계
이이계계계계계계계계계계계계계계계계계계
이이계계계계계계계계계계계계계계계계계계
이이계계배배배배계배배배배배배배배배배배
이이계계배배배배배배배배배배배배배배배배
이이계계배배배배배배배배배배배배배배배배
이이계배배배배배배배배배배배배배배배배배
이이계배배배배배배배배배배배배배배배배배
이이계배배배배배배배배배배배배배배배배배
이이계배배배배배배배배배배배배배배배배배
이이배배배배배배배배배배배배배배배배배배
이이배배배배배배배배배배배배배배배배배배
이이배배배배배배배배배배배배배배배배배배
이이배배배배배배배배배배배배배배배배배배
이이배배배배배배배배배배배배배배배배배배
이이배배배배배배배배배배배배배배배배배배
이이배배배배배배배배배배배배배배배배배배
이이배배배배배배배배배배배배배배배배배배
이이배배배배배배배배배배배배배배배배배배
이이배배배배배배배배배배배배배배배배배배
이이배배배배배배배배배배배배배배배배배배
이이배배배배배배배배배배배배배배배배배배
이이배배배배배배배배배배배배배배배배배배
이이배배배배배배배배배배배배배배배배배배
이이배배배배배배배배배배배배배배배배배배
이이배배배배배배배배배배배배배배배배배배
이이배배배배배배배배배배배배배배배배배배
이이배배배배배배배배배배배배배배배배배배
이이이배이배배이배배이배배배이이배배배배
이이이배이배이이배배이배배이이이이배배이
이이이배이배이이배이이배이이이이이배배이
이이이이이이이이배이이이이이이이이이이이
이이이이이이이이이이이이이이이이이이이이
이이이이이이이이이이이이이이이이이이이이
이이이이이이이이이이이이이

In [261]:
predict_str

'프를를를이어프이계프을이이고프를이이고을'

##7. 불용어처리 + baseline + lr 0.05

In [262]:
sentence = ("배가 고프다"
            "그래서 김치찌개를 먹을 것이다"
            "김치찌개랑 밥이랑 계란을 같이 먹어야지")

In [263]:
world_set = list(set(sentence))
vocab = {char: idx for idx, char in enumerate(world_set)}

In [264]:
print(vocab)

{'먹': 0, '고': 1, '치': 2, '을': 3, '계': 4, '프': 5, '개': 6, '란': 7, '야': 8, '를': 9, '같': 10, '이': 11, '어': 12, '찌': 13, '서': 14, '가': 15, '배': 16, '다': 17, '것': 18, '그': 19, '김': 20, '밥': 21, ' ': 22, '래': 23, '랑': 24, '지': 25}


적절하지 않음 -> 형태소 토큰화

In [265]:
from konlpy.tag import Mecab
tokenizer = Mecab()
print(tokenizer.morphs(sentence))

['배', '가', '고프', '다', '그래서', '김치찌개', '를', '먹', '을', '것', '이', '다', '김치찌개', '랑', '밥', '이랑', '계란', '을', '같이', '먹', '어야지']


In [266]:
tokenized = tokenizer.morphs(sentence)

In [267]:
# 불용어 정의
stopwords=['가','다','를','을','이','다','랑','이랑','어야지','']

In [268]:
tokenized = [word for word in tokenized if word not in stopwords] #불용어 제거

In [269]:
print(tokenized)

['배', '고프', '그래서', '김치찌개', '먹', '것', '김치찌개', '밥', '계란', '같이', '먹']


In [270]:
#단어 집합 생성
word_to_idx = {word: idx for idx, word in enumerate(set(tokenized))} #tokenized에 포함된 단어들을중복 제거 -> 단어를 key, 인덱스를 값으로 사용하여 매핑
idx_to_word = {idx: word for word, idx in word_to_idx.items()} #word to idx 키와 값 순서 바꿔서 변환 매핑

In [271]:
#데이터 생성
input= []
target = []

sequence_length = 3 #문장이 짧아서 짧게 설정
learning_rate = 0.05
vocab_size = len(word_to_idx)
hidden_size = vocab_size

for i in range(len(tokenized) - sequence_length):
  input_seq = tokenized[i:i+sequence_length]
  target_seq = tokenized[i+1:i+sequence_length+1]

  input.append([word_to_idx[word] for word in input_seq])
  target.append([word_to_idx[word] for word in target_seq])

In [272]:
input = np.array(input)
target = np.array(target)

#one-hot encoding
def one_hot_encoding(data,num_classes):
  num_data = data.shape[0]
  num_seq = data.shape[1]
  one_hot = np.zeros((num_data,num_seq,num_classes), dtype=np.float32)
  for i in range(num_data):
    for j in range(num_seq):
      one_hot[i,j,data[i,j]] = 1
  return one_hot

In [273]:
x_one_hot = one_hot_encoding(input, vocab_size)

X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(target)

In [274]:
x_one_hot[0]

array([[0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0.]], dtype=float32)

In [275]:
#RNN (맨 위 베이스라인 참고)

class Net(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, layers):
        super(Net, self).__init__()
        self.rnn1 = torch.nn.RNN(input_size=input_dim, hidden_size=hidden_dim, num_layers=layers, batch_first=True)
        self.rnn2 = torch.nn.RNN(input_size=hidden_dim, hidden_size=hidden_dim, num_layers=layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        out, _ = self.rnn1(x)
        out, _ = self.rnn2(out)
        out = self.fc(out)
        return out

In [276]:
net = Net(vocab_size, hidden_size, 2) #layer=2개

In [277]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

In [278]:
outputs = net(X)
print(outputs.shape)

torch.Size([8, 3, 9])


In [279]:
#학습
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    loss = criterion(outputs.view(-1, vocab_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([world_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += world_set[result[-1]]

    print(predict_str)

프프고고고을을을고고
개개개개개개개개개개
프야야야프프프야먹야
야치야야야야야야먹야
야치야야치야야야먹야
야치야먹치야먹치먹야
야을야먹치야먹개을야
개을야먹치야먹개을야
개란야먹치야프개을야
개란야먹치야프개을야
개란야먹치야프개을야
개란야먹치야프개을야
개란야먹치야프개을먹
란란야먹치야프개을먹
개란야먹치야프개을먹
개란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹

In [280]:
predict_str

'계란야먹치야프개을먹'

##8. 불용어처리 + baseline + lr 0.05 + optimizer-sgd

=> 너무 별로임

In [281]:
sentence = ("배가 고프다"
            "그래서 김치찌개를 먹을 것이다"
            "김치찌개랑 밥이랑 계란을 같이 먹어야지")

In [282]:
world_set = list(set(sentence))
vocab = {char: idx for idx, char in enumerate(world_set)}

In [283]:
print(vocab)

{'먹': 0, '고': 1, '치': 2, '을': 3, '계': 4, '프': 5, '개': 6, '란': 7, '야': 8, '를': 9, '같': 10, '이': 11, '어': 12, '찌': 13, '서': 14, '가': 15, '배': 16, '다': 17, '것': 18, '그': 19, '김': 20, '밥': 21, ' ': 22, '래': 23, '랑': 24, '지': 25}


적절하지 않음 -> 형태소 토큰화

In [284]:
from konlpy.tag import Mecab
tokenizer = Mecab()
print(tokenizer.morphs(sentence))

['배', '가', '고프', '다', '그래서', '김치찌개', '를', '먹', '을', '것', '이', '다', '김치찌개', '랑', '밥', '이랑', '계란', '을', '같이', '먹', '어야지']


In [285]:
tokenized = tokenizer.morphs(sentence)

In [286]:
# 불용어 정의
stopwords=['가','다','를','을','이','다','랑','이랑','어야지','']

In [287]:
tokenized = [word for word in tokenized if word not in stopwords] #불용어 제거

In [288]:
print(tokenized)

['배', '고프', '그래서', '김치찌개', '먹', '것', '김치찌개', '밥', '계란', '같이', '먹']


In [289]:
#단어 집합 생성
word_to_idx = {word: idx for idx, word in enumerate(set(tokenized))} #tokenized에 포함된 단어들을중복 제거 -> 단어를 key, 인덱스를 값으로 사용하여 매핑
idx_to_word = {idx: word for word, idx in word_to_idx.items()} #word to idx 키와 값 순서 바꿔서 변환 매핑

In [290]:
#데이터 생성
input= []
target = []

sequence_length = 3 #문장이 짧아서 짧게 설정
learning_rate = 0.05
vocab_size = len(word_to_idx)
hidden_size = vocab_size

for i in range(len(tokenized) - sequence_length):
  input_seq = tokenized[i:i+sequence_length]
  target_seq = tokenized[i+1:i+sequence_length+1]

  input.append([word_to_idx[word] for word in input_seq])
  target.append([word_to_idx[word] for word in target_seq])

In [291]:
input = np.array(input)
target = np.array(target)

#one-hot encoding
def one_hot_encoding(data,num_classes):
  num_data = data.shape[0]
  num_seq = data.shape[1]
  one_hot = np.zeros((num_data,num_seq,num_classes), dtype=np.float32)
  for i in range(num_data):
    for j in range(num_seq):
      one_hot[i,j,data[i,j]] = 1
  return one_hot

In [292]:
x_one_hot = one_hot_encoding(input, vocab_size)

X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(target)

In [293]:
x_one_hot[0]

array([[0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0.]], dtype=float32)

In [294]:
#RNN (맨 위 베이스라인 참고)

class Net(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, layers):
        super(Net, self).__init__()
        self.rnn1 = torch.nn.RNN(input_size=input_dim, hidden_size=hidden_dim, num_layers=layers, batch_first=True)
        self.rnn2 = torch.nn.RNN(input_size=hidden_dim, hidden_size=hidden_dim, num_layers=layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        out, _ = self.rnn1(x)
        out, _ = self.rnn2(out)
        out = self.fc(out)
        return out

In [295]:
net = Net(vocab_size, hidden_size, 2) #layer=2개

In [296]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), learning_rate)

In [297]:
outputs = net(X)
print(outputs.shape)

torch.Size([8, 3, 9])


In [298]:
#학습
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    loss = criterion(outputs.view(-1, vocab_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([world_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += world_set[result[-1]]

    print(predict_str)

먹먹을먹먹먹먹먹을먹
먹먹먹먹먹먹먹먹먹먹
먹먹먹먹먹먹먹먹먹먹
먹먹먹먹먹먹먹먹먹먹
먹먹먹먹먹먹먹먹먹먹
먹먹먹먹먹먹먹먹먹먹
먹먹먹먹먹먹먹먹먹먹
먹먹먹먹먹먹먹먹먹먹
먹먹먹먹먹먹먹먹먹먹
먹먹먹먹먹먹먹먹먹먹
먹먹먹먹먹먹먹먹먹먹
먹먹먹먹먹먹먹먹먹먹
먹먹먹먹먹먹먹먹먹먹
먹먹먹먹먹먹먹먹먹먹
먹먹먹먹먹먹먹먹먹먹
먹먹먹먹먹먹먹먹먹먹
먹먹먹먹먹먹먹먹먹먹
먹먹먹먹먹먹먹먹먹먹
먹먹먹먹먹먹먹먹먹먹
먹먹먹먹먹먹먹먹먹먹
먹먹먹먹먹먹먹먹먹먹
먹먹먹먹먹먹먹먹먹먹
먹먹먹먹먹먹먹먹먹먹
먹먹먹먹먹먹먹먹먹먹
먹먹먹먹먹먹먹먹먹먹
먹먹먹먹먹먹먹먹먹먹
먹먹먹먹먹먹먹먹먹먹
먹먹먹먹먹먹먹먹먹먹
먹먹먹먹먹먹먹먹먹먹
먹먹먹먹먹먹먹먹먹먹
먹먹야먹먹먹먹먹먹먹
먹먹야먹먹먹먹먹야야
먹먹야먹야야먹먹야야
먹야야야야야먹야야야
먹야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야
야야야야야야야야야야

In [299]:
predict_str

'야야야야야야야야야야'

##9. 불용어처리 + baseline + lr 0.05 + optimizer-sgd + epoch 늘림

In [300]:
sentence = ("배가 고프다"
            "그래서 김치찌개를 먹을 것이다"
            "김치찌개랑 밥이랑 계란을 같이 먹어야지")

In [301]:
world_set = list(set(sentence))
vocab = {char: idx for idx, char in enumerate(world_set)}

In [302]:
print(vocab)

{'먹': 0, '고': 1, '치': 2, '을': 3, '계': 4, '프': 5, '개': 6, '란': 7, '야': 8, '를': 9, '같': 10, '이': 11, '어': 12, '찌': 13, '서': 14, '가': 15, '배': 16, '다': 17, '것': 18, '그': 19, '김': 20, '밥': 21, ' ': 22, '래': 23, '랑': 24, '지': 25}


적절하지 않음 -> 형태소 토큰화

In [303]:
from konlpy.tag import Mecab
tokenizer = Mecab()
print(tokenizer.morphs(sentence))

['배', '가', '고프', '다', '그래서', '김치찌개', '를', '먹', '을', '것', '이', '다', '김치찌개', '랑', '밥', '이랑', '계란', '을', '같이', '먹', '어야지']


In [304]:
tokenized = tokenizer.morphs(sentence)

In [305]:
# 불용어 정의
stopwords=['가','다','를','을','이','다','랑','이랑','어야지','']

In [306]:
tokenized = [word for word in tokenized if word not in stopwords] #불용어 제거

In [308]:
print(tokenized)

['배', '고프', '그래서', '김치찌개', '먹', '것', '김치찌개', '밥', '계란', '같이', '먹']


In [309]:
#단어 집합 생성
word_to_idx = {word: idx for idx, word in enumerate(set(tokenized))} #tokenized에 포함된 단어들을중복 제거 -> 단어를 key, 인덱스를 값으로 사용하여 매핑
idx_to_word = {idx: word for word, idx in word_to_idx.items()} #word to idx 키와 값 순서 바꿔서 변환 매핑

In [310]:
#데이터 생성
input= []
target = []

sequence_length = 3 #문장이 짧아서 짧게 설정
learning_rate = 0.05
vocab_size = len(word_to_idx)
hidden_size = vocab_size

for i in range(len(tokenized) - sequence_length):
  input_seq = tokenized[i:i+sequence_length]
  target_seq = tokenized[i+1:i+sequence_length+1]

  input.append([word_to_idx[word] for word in input_seq])
  target.append([word_to_idx[word] for word in target_seq])

In [311]:
input = np.array(input)
target = np.array(target)

#one-hot encoding
def one_hot_encoding(data,num_classes):
  num_data = data.shape[0]
  num_seq = data.shape[1]
  one_hot = np.zeros((num_data,num_seq,num_classes), dtype=np.float32)
  for i in range(num_data):
    for j in range(num_seq):
      one_hot[i,j,data[i,j]] = 1
  return one_hot

In [312]:
x_one_hot = one_hot_encoding(input, vocab_size)

X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(target)

In [313]:
x_one_hot[0]

array([[0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0.]], dtype=float32)

In [314]:
#RNN (맨 위 베이스라인 참고)

class Net(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, layers):
        super(Net, self).__init__()
        self.rnn1 = torch.nn.RNN(input_size=input_dim, hidden_size=hidden_dim, num_layers=layers, batch_first=True)
        self.rnn2 = torch.nn.RNN(input_size=hidden_dim, hidden_size=hidden_dim, num_layers=layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        out, _ = self.rnn1(x)
        out, _ = self.rnn2(out)
        out = self.fc(out)
        return out

In [315]:
net = Net(vocab_size, hidden_size, 2) #layer=2개

In [316]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), learning_rate)

In [317]:
outputs = net(X)
print(outputs.shape)

torch.Size([8, 3, 9])


In [320]:
#학습
for i in range(1000):
    optimizer.zero_grad()
    outputs = net(X)
    loss = criterion(outputs.view(-1, vocab_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([world_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += world_set[result[-1]]

    print(predict_str)

계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹

In [321]:
predict_str

'계란야먹치야프개을먹'

##10. 불용어처리 + baseline + lr 0.05 + epoch늘림

In [322]:
sentence = ("배가 고프다"
            "그래서 김치찌개를 먹을 것이다"
            "김치찌개랑 밥이랑 계란을 같이 먹어야지")

In [323]:
world_set = list(set(sentence))
vocab = {char: idx for idx, char in enumerate(world_set)}

In [324]:
print(vocab)

{'먹': 0, '고': 1, '치': 2, '을': 3, '계': 4, '프': 5, '개': 6, '란': 7, '야': 8, '를': 9, '같': 10, '이': 11, '어': 12, '찌': 13, '서': 14, '가': 15, '배': 16, '다': 17, '것': 18, '그': 19, '김': 20, '밥': 21, ' ': 22, '래': 23, '랑': 24, '지': 25}


적절하지 않음 -> 형태소 토큰화

In [325]:
from konlpy.tag import Mecab
tokenizer = Mecab()
print(tokenizer.morphs(sentence))

['배', '가', '고프', '다', '그래서', '김치찌개', '를', '먹', '을', '것', '이', '다', '김치찌개', '랑', '밥', '이랑', '계란', '을', '같이', '먹', '어야지']


In [326]:
tokenized = tokenizer.morphs(sentence)

In [327]:
# 불용어 정의
stopwords=['가','다','를','을','이','다','랑','이랑','어야지','']

In [328]:
tokenized = [word for word in tokenized if word not in stopwords] #불용어 제거

In [329]:
print(tokenized)

['배', '고프', '그래서', '김치찌개', '먹', '것', '김치찌개', '밥', '계란', '같이', '먹']


In [330]:
#단어 집합 생성
word_to_idx = {word: idx for idx, word in enumerate(set(tokenized))} #tokenized에 포함된 단어들을중복 제거 -> 단어를 key, 인덱스를 값으로 사용하여 매핑
idx_to_word = {idx: word for word, idx in word_to_idx.items()} #word to idx 키와 값 순서 바꿔서 변환 매핑

In [331]:
#데이터 생성
input= []
target = []

sequence_length = 3 #문장이 짧아서 짧게 설정
learning_rate = 0.05
vocab_size = len(word_to_idx)
hidden_size = vocab_size

for i in range(len(tokenized) - sequence_length):
  input_seq = tokenized[i:i+sequence_length]
  target_seq = tokenized[i+1:i+sequence_length+1]

  input.append([word_to_idx[word] for word in input_seq])
  target.append([word_to_idx[word] for word in target_seq])

In [332]:
input = np.array(input)
target = np.array(target)

#one-hot encoding
def one_hot_encoding(data,num_classes):
  num_data = data.shape[0]
  num_seq = data.shape[1]
  one_hot = np.zeros((num_data,num_seq,num_classes), dtype=np.float32)
  for i in range(num_data):
    for j in range(num_seq):
      one_hot[i,j,data[i,j]] = 1
  return one_hot

In [333]:
x_one_hot = one_hot_encoding(input, vocab_size)

X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(target)

In [334]:
x_one_hot[0]

array([[0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0.]], dtype=float32)

In [335]:
#RNN (맨 위 베이스라인 참고)

class Net(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, layers):
        super(Net, self).__init__()
        self.rnn1 = torch.nn.RNN(input_size=input_dim, hidden_size=hidden_dim, num_layers=layers, batch_first=True)
        self.rnn2 = torch.nn.RNN(input_size=hidden_dim, hidden_size=hidden_dim, num_layers=layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        out, _ = self.rnn1(x)
        out, _ = self.rnn2(out)
        out = self.fc(out)
        return out

In [336]:
net = Net(vocab_size, hidden_size, 2) #layer=2개

In [337]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

In [338]:
outputs = net(X)
print(outputs.shape)

torch.Size([8, 3, 9])


In [339]:
#학습
for i in range(1000):
    optimizer.zero_grad()
    outputs = net(X)
    loss = criterion(outputs.view(-1, vocab_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([world_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += world_set[result[-1]]

    print(predict_str)

란란란란란란란란란란
먹먹먹먹먹먹먹먹먹먹
먹먹먹먹먹먹먹먹먹먹
야야야야야야야야야야
야야야야야야야야야야
치야야먹야야프야야먹
치야야먹야야프개먹먹
치프개먹먹야프개먹먹
치프개먹치야프개먹먹
치프야먹치야야개을먹
치프야먹치야야개을먹
치프야먹치야야개을먹
치프야먹치야야개을먹
치프야야치야야개을먹
치프야먹치야야개을먹
치란야먹치야야개을먹
치란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹
계란야먹치야프개을먹

In [340]:
predict_str

'계란야먹치야프개을먹'